In [2]:
import json
import csv
import pandas as pd
import sqlite3
import networkx as nx
import matplotlib as plt
import numpy as np

In [3]:
conn = sqlite3.connect('nft_database_simple.db')
c = conn.cursor()

In [4]:
df_all_transactions = pd.read_sql_query("SELECT * from all_transactions", conn)
df_all_transactions.head()

collection_id collection_name  token_id            block_timestamp  \
0              6       Cool Cats         0  2021-07-03 17:34:34+00:00   
1              6       Cool Cats         0  2021-12-31 04:57:23+00:00   
2              6       Cool Cats         4  2021-07-04 19:37:16+00:00   
3              6       Cool Cats         4  2022-02-14 12:30:16+00:00   
4              6       Cool Cats         4  2022-03-07 01:02:23+00:00   

                                 from_address  \
0  0xfc86a64a8de22cf25410f7601acbd8d6630da93d   
1  0x98bb3a3921200017ebd0af803aeddd464e70e791   
2  0x53722a32fddda871f35ef628b252d349744d4b71   
3  0x433c8a73bec1273561e4e2201649de12f20b7d58   
4  0x4d6d847627b96ef00767e0984e10e8e1d884fcf4   

                                   to_address  
0  0x98bb3a3921200017ebd0af803aeddd464e70e791  
1  0xd9463b55af194d4fa22a532a52f5bdd45b66d290  
2  0x433c8a73bec1273561e4e2201649de12f20b7d58  
3  0x4d6d847627b96ef00767e0984e10e8e1d884fcf4  
4  0x6d8de2f66efcc3af613e168d98e41769c4357696

In [5]:
df_collections = pd.read_sql_query("SELECT * from collections", conn)
df_collections

collection_id                             collection_hash  \
0               1  0x7bd29408f11d2bfc23c34f18275bbf23bb716bc7   
1               2  0x4e1f41613c9084fdb9e34e11fae9412427480e56   
2               3  0xff9c1b15b16263c61d017ee9f65c50e4ae0113d7   
3               4  0xce25e60a89f200b1fa40f6c313047ffe386992c3   
4               5  0x49cf6f5d44e70224e2e23fdcdd2c053f30ada28b   
5               6  0x1a92f7381b9f03921564a437210bb9396471050c   
6               7  0xed5af388653567af2f388e6224dc7c4b3241c544   
7               8  0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb   
8               9  0x8a90cab2b38dba80c64b7734e58ee1db38b8992e   
9              10  0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d   
10             11  0x60e4d786628fea6478f785a6d7e704777c86a7c6   
11             12  0xba30e5f9bb24caa003e9f2f0497ad287fdf95623   
12             13  0xe785e82358879f061bc3dcac6f0444462d4b5330   
13             14  0x1cb1a5e65610aeff2551a50f76a87a7d3fb649c6   

          collection_name  abnormal_index  
0                 Meebits        1.820205  
1               Terraform       12.941688  
2                    Loot        5.462385  
3              dotdotdots        0.571304  
4                  CloneX        0.415401  
5               Cool Cats        1.256944  
6                   Azuki        0.256567  
7             Cryptopunks       16.269810  
8                 Doodles        3.682270  
9    Bored Ape Yacht Club        1.939159  
10  Mutant Ape Yacht Club        0.475361  
11  Bored Ape Kennel Club        0.424828  
12         World of Women        1.253759  
13              CrypToadz        0.661297

In [56]:
df_tokens = pd.read_sql_query("SELECT * from tokens_by_num_trans", conn)
df_tokens.head()

collection_name  token_id  count
0       Terraform      1865   1151
1       Terraform      9559   1055
2       Terraform      5283   1050
3       Terraform      9728    991
4         Meebits      8319    728

In [111]:
df_tokens_by_collection["sum"] = df_tokens.groupby(["collection_name","token_id"]).sum()
df_tokens_by_collection = df_tokens_by_collection.sort_values(["collection_name","sum"],ascending=False).groupby(["collection_name"]).head(20)
# df_tokens_by_collection = df_tokens_by_collection.drop(values)
df_tokens_by_collection

count  sum
collection_name token_id            
dotdotdots      1948        475  475
                2211        154  154
                2682        102  102
                3521         89   89
                4477         80   80
...                         ...  ...
Azuki           4882         16   16
                7258         16   16
                1587         15   15
                4578         15   15
                6084         15   15

[280 rows x 2 columns]

In [118]:
indexes = []
cols = []
tokens = []

In [120]:
for collection,token in df_tokens_by_collection.index:
    # get token
    temp_token = df_all_transactions[(df_all_transactions["collection_name"] == collection)
                                     & (df_all_transactions["token_id"] == token)]
    
    # get total number of transactions
    total_transaction_num = temp_token.shape[0]
    
    # get total number of distinct wallets
    df_distinct_wallets = temp_token['from_address'].append(temp_token['to_address'])
    total_distinct_wallets = df_distinct_wallets.unique().shape[0]
    
    # transaction num by distinct wallet
    df_transaction_num_dist = df_distinct_wallets.value_counts()
    df_transaction_num_dist = df_transaction_num_dist.sort_values(ascending=False)
    df_transaction_num_dist = pd.DataFrame({'wallet_hash':df_transaction_num_dist.index, collection:df_transaction_num_dist.values})
    
    # get transaction num distribution (by frequency)
    temp_freq = df_transaction_num_dist
    temp_freq['freq'] = df_transaction_num_dist.value_counts().values
    temp_freq_group = temp_freq.groupby(collection).sum('freq')
    temp_freq_group[collection] = temp_freq_group.index
    
#     print(temp_freq_group)
    
    # calculate normalized skewness of transaction distribution
    skewness = 3*(temp_freq_group['freq'].mean()-temp_freq_group['freq'].median())/temp_freq_group[collection].std()
    
    # calculate abnormal index
    if (skewness == 0 or total_distinct_wallets == 0):
        continue
    index = total_transaction_num/(total_distinct_wallets*skewness)
    
    if (index == np.inf or index == -np.inf):
        continue
    
    # append index
    indexes.append(index)
    cols.append(collection)
    tokens.append(token)
    

In [121]:
print(cols)

['dotdotdots', 'dotdotdots', 'dotdotdots', 'dotdotdots', 'dotdotdots', 'dotdotdots', 'dotdotdots', 'dotdotdots', 'dotdotdots', 'dotdotdots', 'dotdotdots', 'dotdotdots', 'dotdotdots', 'World of Women', 'World of Women', 'World of Women', 'World of Women', 'World of Women', 'World of Women', 'World of Women', 'World of Women', 'World of Women', 'World of Women', 'World of Women', 'World of Women', 'World of Women', 'World of Women', 'World of Women', 'Terraform', 'Terraform', 'Terraform', 'Terraform', 'Terraform', 'Terraform', 'Terraform', 'Terraform', 'Terraform', 'Terraform', 'Terraform', 'Terraform', 'Terraform', 'Mutant Ape Yacht Club', 'Mutant Ape Yacht Club', 'Mutant Ape Yacht Club', 'Mutant Ape Yacht Club', 'Mutant Ape Yacht Club', 'Mutant Ape Yacht Club', 'Mutant Ape Yacht Club', 'Mutant Ape Yacht Club', 'Mutant Ape Yacht Club', 'Mutant Ape Yacht Club', 'Mutant Ape Yacht Club', 'Mutant Ape Yacht Club', 'Mutant Ape Yacht Club', 'Mutant Ape Yacht Club', 'Mutant Ape Yacht Club', 'Mu

In [122]:
df_result = pd.DataFrame({'collection_name': cols,
                          'token_id': tokens,
                          'abnormal_index':indexes})
df_result = df_result.sort_values(by=["collection_name","abnormal_index"],ascending=False)
df_result
# print(df_result[df_result['collection_name'] == "Terraform"])

collection_name  token_id  abnormal_index
0        dotdotdots      1948     4251.772110
1        dotdotdots      2211     1472.019737
6        dotdotdots      4638      535.401545
2        dotdotdots      2682      355.908945
10       dotdotdots      4428      119.942437
..              ...       ...             ...
233           Azuki      2049        0.152753
226           Azuki       699        0.147035
232           Azuki      8742        0.138866
224           Azuki      7569        0.095470
222           Azuki      1717        0.015275

[241 rows x 3 columns]

In [128]:
df_result[df_result['collection_name'] == "World of Women"]

collection_name  token_id  abnormal_index
16  World of Women      5375       38.896786
18  World of Women      9459       38.896786
21  World of Women      1948       29.662796
23  World of Women      4878       29.662796
13  World of Women      8124       14.556440
14  World of Women      8886       10.290155
19  World of Women      1315        5.894398
20  World of Women      1422        5.894398
15  World of Women      5097        4.782782
22  World of Women      4358        2.206695
25  World of Women      7215        1.908111
27  World of Women       326        1.885618
17  World of Women      7858        0.826028
24  World of Women      6954        0.727748
26  World of Women      9358        0.725731

In [127]:
df_result.to_sql("tokens_fraud_index", conn, if_exists='replace', index=False)